# Project 4 - West Nile Virus Prediction

## Problem Statement
The intend of this project is to analyze weather data and GIS data and predicting whether or not West Nile virus is present, for a given time, location, and species. 

##  General Approach
- [Data cleaning](#Data-Cleaning)
- [Data imputation](#Data-Imputation)
- [Data visualization](#Data-Visualization)
- [Feature selection](#Featuion-Selection)
- [Cross validation](#Cross-Validation)
- [Model fitting](#Model-Fitting)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

In [2]:
# Load dataset 
train = pd.read_csv('assets/train.csv')
test = pd.read_csv('assets/test.csv')
sample = pd.read_csv('assets/sampleSubmission.csv')
weather = pd.read_csv('assets/weather.csv')
spray = pd.read_csv('assets/spray.csv')

### Data Cleaning

#### Cleaning train and test data

In [3]:
train.head(3)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,T007,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0


In [4]:
test.head()

,Id,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy
0,1,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
1,2,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
2,3,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
3,4,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX SALINARIUS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9
4,5,2008-06-11,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX TERRITANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9


In [5]:
#Define function to get year and month from date
get_year = lambda x:x.split('-')[0]
get_month = lambda x:x.split('-')[1]

In [6]:
# Convert to date time format
train['Date'] = pd.to_datetime(train['Date'])
test['Date'] = pd.to_datetime(test['Date'])

In [7]:
#Find Satellite Trap in Train Data
satellite_trap = [t for t in train['Trap'].value_counts().index if len(t) == 5]
satellite_trap

['T054C', 'T094B']

In [8]:
#Find extra categories in Trap in Test data

extra = [t for t in test['Trap'].value_counts().index if t not in train['Trap'].value_counts().index]
extra

['T090A',
 'T065A',
 'T200A',
 'T218B',
 'T218C',
 'T218A',
 'T234',
 'T200B',
 'T090B',
 'T090C',
 'T002B',
 'T002A',
 'T128A']

'T234' is present in df_test['Trap'] but not df_train['Trap']

In [9]:
#convert all satellite trap to have the same trap name as parent trap
trap_rename = lambda x: x[0:4] if len(x)==5 else x
train['Trap'] = train['Trap'].apply(trap_rename)
test['Trap'] = test['Trap'].apply(trap_rename)

In [10]:
#Remove T in Trap name
remove_t = lambda x: x.replace('T','')
train['Trap'] = train['Trap'].apply(remove_t)
test['Trap'] = test['Trap'].apply(remove_t)

#Convert Trap type to int
train['Trap'] = train['Trap'].astype('int64')
test['Trap'] = test['Trap'].astype('int64')

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10506 entries, 0 to 10505
Data columns (total 12 columns):
Date                      10506 non-null datetime64[ns]
Address                   10506 non-null object
Species                   10506 non-null object
Block                     10506 non-null int64
Street                    10506 non-null object
Trap                      10506 non-null int64
AddressNumberAndStreet    10506 non-null object
Latitude                  10506 non-null float64
Longitude                 10506 non-null float64
AddressAccuracy           10506 non-null int64
NumMosquitos              10506 non-null int64
WnvPresent                10506 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 985.1+ KB


In [12]:
train.head()

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.954690,-87.800991,9,1,0
2,2007-05-29,"6200 North Mandell Avenue, Chicago, IL 60646, USA",CULEX RESTUANS,62,N MANDELL AVE,7,"6200 N MANDELL AVE, Chicago, IL",41.994991,-87.769279,9,1,0
3,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX PIPIENS/RESTUANS,79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,1,0
4,2007-05-29,"7900 West Foster Avenue, Chicago, IL 60656, USA",CULEX RESTUANS,79,W FOSTER AVE,15,"7900 W FOSTER AVE, Chicago, IL",41.974089,-87.824812,8,4,0


In [13]:
#Each row only has maximum 50 mosquito before a new row is created, need to sum up the total mosquito by species,
#trap,date and WnvPresent
train.groupby(['Date', 'Trap', 'Species','WnvPresent'], as_index=False)['NumMosquitos'].sum().sort_values(by='NumMosquitos')


,Date,Trap,Species,WnvPresent,NumMosquitos
0,2007-05-29,2,CULEX PIPIENS/RESTUANS,0,1
4642,2009-09-25,128,CULEX PIPIENS,0,1
1806,2007-08-24,34,CULEX PIPIENS,0,1
4639,2009-09-25,115,CULEX PIPIENS,0,1
4638,2009-09-25,114,CULEX PIPIENS/RESTUANS,0,1
...,...,...,...,...,...
860,2007-08-01,115,CULEX PIPIENS,0,1543
1220,2007-08-08,138,CULEX PIPIENS,0,1558
1481,2007-08-16,115,CULEX PIPIENS,0,2050
1954,2007-08-28,115,CULEX PIPIENS,0,2107


In [14]:
#merge total mosquito with train data
train = pd.merge(train, train.groupby(['Date', 'Trap', 'Species','WnvPresent'], as_index=False)['NumMosquitos'].sum(), 
                    how='left', on=['Date', 'Trap', 'Species','WnvPresent'])
#rename column wich include total mosquito
train.columns = ['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent','Tot_Mos_Species']
train.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,Tot_Mos_Species
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,2,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,1


In [15]:
train = train[train.duplicated(subset=['Date', 'Trap', 'Species','Tot_Mos_Species','WnvPresent']) == False]
len(train)

8599

In [16]:
# remove redundant species name
train['Species'] = train['Species'].apply(lambda x:x.replace('CULEX',''))
test['Species'] = test['Species'].apply(lambda x:x.replace('CULEX',''))

In [17]:
# Grouping minority species as others 
train[['Species','WnvPresent']].groupby('Species').count()

,WnvPresent
Species,
ERRATICUS,1
PIPIENS,1994
PIPIENS/RESTUANS,3893
RESTUANS,2406
SALINARIUS,83
TARSALIS,6
TERRITANS,216


In [18]:
# Grouping minority species as others 

others = [' ERRATICUS',' SALINARIUS',' TARSALIS',' TERRITANS']
train['Species'] = train['Species'].apply(lambda x: 'Others' if x in others else x.replace(' ',''))
test['Species'] = test['Species'].apply(lambda x: 'Others' if x in others else x.replace(' ',''))

#### Cleaning Spray data

#### Cleaning Weather data.
We will look through the weather dataset to see if we can reasonably fill the missing values and drop columns which we feel are less or not relevant to help us predict the presence of the mosquito or virus.

In [20]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


After going through the columns value counts, we derived that these features cols are not important as they have low variance

In [21]:
weather.columns

Index(['Station', 'Date', 'Tmax', 'Tmin', 'Tavg', 'Depart', 'DewPoint',
       'WetBulb', 'Heat', 'Cool', 'Sunrise', 'Sunset', 'CodeSum', 'Depth',
       'Water1', 'SnowFall', 'PrecipTotal', 'StnPressure', 'SeaLevel',
       'ResultSpeed', 'ResultDir', 'AvgSpeed'],
      dtype='object')

In [22]:
weather_cols_drop = ['Depart', 'SunRise', 'SunSet', 'Depth', 'Water1', 'SnowFall']

In [23]:
# looking at the data types
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg            object
Depart          object
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

Some observations from looking at the weather types
- 'Tavg' should have a daya type of integer or float since it is the average of 2 integers ('Tmax' and 'Tmin'). 
- Features like 'Depart', 'Sunrise', 'Sunset', 'Depth', 'Water1', 'Snowfall', 'PrecipTotal', 'StnPressure', 'SeaLevel', 'AvgSpeed' should also be either integer or float. 

In [24]:
# checking the values for 'Depart'
weather[weather['Depart'] == 'M'].head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
5,2,2007-05-03,67,48,58,M,40,50,7,0,...,HZ,M,M,M,0.00,29.46,30.12,12.9,6,13.2
7,2,2007-05-04,78,51,M,M,42,50,M,M,...,,M,M,M,0.00,29.36,30.04,10.1,7,10.4
9,2,2007-05-05,66,54,60,M,39,50,5,0,...,,M,M,M,T,29.46,30.09,11.2,7,11.5


In [25]:
# we will be dropping 'Depart' as all 'Depart' entries are missing from Station 2
weather.drop(columns='Depart', inplace=True)

In [26]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,51,56,0,2,0448,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,51,57,0,3,-,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,42,47,14,0,0447,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,42,47,13,0,-,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,40,48,9,0,0446,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [27]:
weather['PrecipTotal'] = weather['PrecipTotal'].map(lambda x: '0.00' if 'M' in x else x)
weather['PrecipTotal'] = weather['PrecipTotal'].map(lambda x: '0.00' if 'T' in x else x)

In [28]:
# convert to floating dtype
weather['PrecipTotal'].astype(float)

0       0.00
1       0.00
2       0.00
3       0.00
4       0.00
        ... 
2939    0.00
2940    0.00
2941    0.00
2942    0.03
2943    0.04
Name: PrecipTotal, Length: 2944, dtype: float64

In [29]:
weather.head()

,Station,Date,Tmax,Tmin,Tavg,DewPoint,WetBulb,Heat,Cool,Sunrise,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,51,56,0,2,0448,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,51,57,0,3,-,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,42,47,14,0,0447,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,42,47,13,0,-,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,40,48,9,0,0446,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [30]:
# obtaining Tavg from Tmin and Tmax
weather['Tavg'] = np.round(((weather['Tmax']+ weather['Tmin']) + 1)/2)
weather['Tavg'] = weather['Tavg'].astype(int)

In [31]:
weather.dtypes

Station          int64
Date            object
Tmax             int64
Tmin             int64
Tavg             int64
DewPoint         int64
WetBulb         object
Heat            object
Cool            object
Sunrise         object
Sunset          object
CodeSum         object
Depth           object
Water1          object
SnowFall        object
PrecipTotal     object
StnPressure     object
SeaLevel        object
ResultSpeed    float64
ResultDir        int64
AvgSpeed        object
dtype: object

In [32]:
M_index = weather[weather['AvgSpeed'] == 'M']['AvgSpeed'].index


In [33]:
weather.loc[M_index, 'AvgSpeed'] = np.nan

In [34]:
weather['AvgSpeed'].fillna(weather[['AvgSpeed']].mean())

0        9.2
1        9.6
2       13.4
3       13.4
4       11.9
        ... 
2939     9.0
2940     5.5
2941     6.5
2942    22.9
2943    22.6
Name: AvgSpeed, Length: 2944, dtype: object

In [35]:
weather['AvgSpeed'].value_counts()

6.9     63
5.8     60
7.4     55
8.1     49
7.0     47
        ..
26.3     1
18.2     1
2.4      1
23.2     1
15.8     1
Name: AvgSpeed, Length: 177, dtype: int64

In [36]:
weather['AvgSpeed'] = weather['AvgSpeed'].astype(float)

In [37]:
weather_cols_drop = ['Sunrise', 'Sunset', 'Depth', 'Water1', 'SnowFall']

In [38]:
weather.drop(columns=weather_cols_drop, inplace=True)

In [39]:
weather['Date'] = pd.to_datetime(weather['Date'])

In [40]:
train.columns

Index(['Date', 'Address', 'Species', 'Block', 'Street', 'Trap',
       'AddressNumberAndStreet', 'Latitude', 'Longitude', 'AddressAccuracy',
       'NumMosquitos', 'WnvPresent', 'Tot_Mos_Species'],
      dtype='object')

#### Merging weather data to train/test data

In [41]:
def closest_point(point):
    station1, station2 = [41.995, -87.933], [41.786, -87.752]  # Fixed coordinates for two stations
    points = [station1, station2] 
    if cdist([point], points).argmin() == 0: return 1 # return index of closest point
    return 2

In [42]:
# Assign Station to train data based on station coordinates
train['Station'] = [closest_point(x) for x in train[['Latitude','Longitude']].values]
test['Station'] = [closest_point(x) for x in test[['Latitude','Longitude']].values]

In [43]:
# Merge weather data to train data base on date and station number
train = train.merge(weather, on=['Date','Station'])
test = test.merge(weather, on=['Date','Station'])

#### Merging spray data to train/test data

### Data Visualization

### Feature Selection

### Cross Validation

### Model Fitting